<a href="https://colab.research.google.com/github/SouvikBandyopadhyay/ClassificationModelCompairision/blob/main/MathsAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Name: Souvik Bandyopadhyay
### Enrollment No.: 2023CSM011
### Subject: Maths for CS
### M.Tech 1st Sem, Session 2023-24

# Question
## Assignment 1: 20 marks

Consider any dataset from any online machine learning data repository. Split the data in 80-20 ratio for train and test dataset (can also perform k-fold cross validation if dataset is big enough). Perform feature selection using Core and Reduct (Rough Set Theory). Then perform:
* i. Rule based Classification (using Decision Matrix from Rough Set Theory)
* ii. Decision Tree Classifier
* iii. Naive Bayes Classifier

## Output:
* i. Print Accuracy, Precision, Recall, F1 Score against training dataset.
* ii. Print Accuracy, Precision, Recall, F1 Score against test dataset.


# Answer
# Introduction

* This project aims to compare the performance of 3 classifier model, 1. Decision Rule, 2. Naive Bayes, 3. Decision Tree, and uses concepts of Rough Set for feature selection.



### Imports
`import pandas as pd` , Pandas is a data python library for manipulation and analysis, pandas is used as the primary dataframe management tool in this project.


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Rough Set Theory

* Primarily used for feature selection

* **Class roughSet** contains member variables **"dataset", "targetsets" and "targetset"**

* functions:
* * `ind(dataset,targetset,targetsets)` : used to find **Indiscernibility** on a given Information System and Target Set
* * `lowerbound(target,datasetind)` : used to find lower bound using `ind()`, takes 1D array of object as target
* * `upperbound(target)` : used to find upper bound using `ind()`, takes 1D array of object as target
* * `boundary()` : finds boundary region, i.e. the difference between upper and lower bound
* * `posetiveRegion(targets,datasetind)` : used to find Posetive Region using union of `lowerbound()`, takes 2D array of object as targets
* * `negetiveRegion(targets)` : used to find Negetive Region by substracting union of `upperbound()` from dataset, takes 2D array of object as targets
* * `boundaryRegion(targets)` : finds boundary region, i.e objects in upper union but not in lower union
* * `iscrisp()` : checks if set is crisp

In [ ]:

class roughSet:
    columns1=["A1","A2","D"]
    dataset1={"dataset":[[1,2,0],[1,2,0],[2,0,0],[0,0,1],[2,1,0],[0,0,1],[2,0,0],[0,1,2],[2,1,0],[2,0,0]],
    "targetset":[1,2,3],
    "targetsets":[[1,2,3,10],[4,5,9],[6,7,8]]}

    dataset=[]
    targetset=[]
    targetsets=[]
    # dataframe=pd.DataFrame(dataset1["dataset"],columns=columns1)

    def __init__(self,dataset=dataset1["dataset"],targetset=dataset1["targetset"],targetsets=dataset1["targetsets"]):
        self.dataset=dataset
        self.targetsets=targetsets
        self.targetset=targetset


    def setdataset(self,dataframe):
        global dataset
        dataset=list(zip(*map(dataframe.get,dataframe)))

    def ind(self,dataset=None,p=None):
        if dataset is None:
            dataset=self.dataset
        if p is None:
            p=[x for x in range(len(self.dataset[0]))]
        dic=dict()
        seen=[]
        # p=[0,2]
        # print(p)
        for i in range(len(dataset)):
            setx=[dataset[i][x] for x in range(len(dataset[i])) if x in p]
            # print(seen)
            if setx in seen:
                dic[seen.index(setx)].append(i+1)
            else:
                seen.append(setx)
                dic[seen.index(setx)]=[i+1]

            # print("dict===",dic)
        return list(dic.values())

    def allin(self,x,target):
        for i in x:
            if i not in target:
                return False
        return True

    def lowerbound(self,target=None,datasetind=None):
        if target==None:
            target=self.targetset
        result=[]
        if datasetind==None:
            datasetind=self.ind()
        for i in datasetind:
            if self.allin(i,target):
                [result.append(x) for x in i]
        return set(result)


    def anyin(self,x,target):
        for i in x:
            if i in target:
                return True
        return False

    def upperbound(self,target=None):
        if target==None:
            target=self.targetset
        result=[]
        for i in self.ind():
            if self.anyin(i,target):
                [result.append(x) for x in i]
        return set(result)

    def accuracy(self):
        return len(self.lowerbound())*100/len(self.upperbound())

    def boundary(self):
        lb=self.lowerbound()
        result=[]
        for i in (self.upperbound()):
            if i not in lb:
                result.append(i)
        return set(result)

    def lowerunion(self,targets=None,datasetind=None):
        if targets==None:
            targets=self.targetsets
        result=[]

        for i in targets:
            [result.append(x) for x in self.lowerbound(i,datasetind=datasetind)]
        return set(result)

    def posetiveRegion(self,targets=None,datasetind=None):
        if targets==None:
            targets=self.targetsets
        return self.lowerunion(targets,datasetind=datasetind)

    def upperunion(self,targets=None):
        if targets==None:
            targets=self.targetsets
        result=[]
        for i in targets:
            [result.append(x) for x in self.upperbound(i)]
        return set(result)

    def negetiveRegion(self,targets=None):
        if targets==None:
            targets=self.targetsets
        antiresult=self.upperunion(targets)
        result=[x+1 for x in range(len(dataset)) if (x+1) not in antiresult]
        return set(result)

    def boundaryRegion(self,targets=None):
        if targets==None:
            targets=self.targetsets
        lu=self.lowerunion(targets)
        uu=self.upperunion(targets)
        result=[x for x in uu if x not in lu]
        return set(result)

    def iscrisp(self):
        if len(self.lowerunion())/len(self.upperunion())==1:
            return True
        return False



# Reduct and Core

* **class reductandcore** has member variables **"dataframe"** and **"y"**

* funtion:
* * `core()` : returnes the core of a given dataset using **Discernibility matrix**
* * `reduct()` : returnes the reduct of a given dataset using **Modified Quick Reduct Algorithm** *Frequency of non-core attributes in discernibility matrix*
* * `hReduced()` : returns a Horizontaly reduced dataset using `reduct()`
* * `vReduced()` : returns a Horizontaly and Vertically reduced dataset

In [ ]:
class reductandcore:
    def __init__(self, dataframe=None, y=None):
        if dataframe is None:
            columns = ["A1", "A2", "A3", "A4", "D"]
            dataset = [["M", "L", 3, "M", 1],
                       ["M", "L", 1, "H", 1],
                       ["L", "L", 1, "M", 1],
                       ["L", "L", 1, "M", 2],
                       ["L", "R", 3, "M", 2],
                       ["M", "R", 2, "M", 2],
                       ["L", "R", 3, "L", 3],
                       ["H", "R", 3, "L", 3],
                       ["H", "N", 3, "L", 3]]
            self.dataframe = pd.DataFrame(dataset, columns=columns)
        else:
            self.dataframe = dataframe
        if y is None:
            self.y = "D"
        else:
            self.y = y
        # print(self.dataframe)

    def core(self):
        corematrix = []
        datalist = []
        deletable = list()
        dftemp = self.dataframe.loc[:, self.dataframe.columns != self.y]
        dicforcore = {x: 0 for x in dftemp.columns}
        dicforcoremain = {x: [] for x in range(len(dftemp.columns)+1)}

        # print(len(self.dataframe), self.dataframe[self.y][0])
        for i in range(len(self.dataframe)):
            ithcorematrix = []
            for j in range(i):
                if self.dataframe[self.y][i] == self.dataframe[self.y][j]:
                    ithcorematrix.append([])
                else:
                    mismatchlist = []
                    for k in dftemp.columns:
                        if dftemp[k][i] != dftemp[k][j]:
                            # print(dftemp[k][i], dftemp[k][j], end=" , ")
                            mismatchlist.append(k)
                            dicforcore[k] += 1
                    # print(i, j, mismatchlist)
                    dicforcoremain[len(mismatchlist)].append(mismatchlist)
                    ithcorematrix.append(mismatchlist)
            # print("ithcore", ithcorematrix)

            corematrix.append(ithcorematrix)
        # for i in deletable:

        # print("dictfor core main==", dicforcoremain)
        # print("friquency ===", dicforcore)

        # print("---------------------------Discernibility Matrix------------------------")
        # for i in corematrix:
            # print(i)
        for i in range(len(dftemp.columns)):
            if dicforcoremain[i+1] != []:
                corelist = dicforcoremain[i+1]
                break
        cores = list()
        for i in corelist:
            for j in i:
                cores.append(j)
        cores = set(cores)
        sortedreducts = [x[0] for x in sorted(
            dicforcore.items(), key=lambda x: x[1], reverse=True) if x[0] not in cores]
        return {"core": cores, "atts": sortedreducts}

    def reduct(self):
        coresandatts = self.core()
        cores = coresandatts["core"]
        atts = coresandatts["atts"]
        # print(cores,atts)
        t = cores
        dftemp = self.dataframe.loc[:, self.dataframe.columns != self.y]
        ddtemp = self.dataframe[[self.y]]
        rsdd = roughSet(dataset=ddtemp.values.tolist())
        indD = rsdd.ind()
        # print("ind D=== ",indD)
        rsdf = roughSet(dataset=dftemp.values.tolist())
        indcore = rsdf.ind(
            p=[x for x in range(len(dftemp.columns)) if dftemp.columns[x] in cores])
        # print("ind core==",indcore)
        T = list(cores)
        # print("cores",cores,T)

        lowerunionwithcore = rsdf.posetiveRegion(
            targets=indD, datasetind=indcore)
        # print("Lower union== ",lowerunionwithcore)
        gama = len(lowerunionwithcore)/len(dftemp.index)
        # print("gamma core====",gama)
        for i in atts:
            tempT = T.copy()
            tempT.append(i)
            # print("tempt=",tempT)
            indiT = rsdf.ind(
                p=[x for x in range(len(dftemp.columns)) if dftemp.columns[x] in tempT])
            # print("ind ====== T ",T," is ",indiT)
            lowerunion = rsdf.posetiveRegion(targets=indD, datasetind=indiT)
            # print("Lower union== ",lowerunion)
            gamatemp = len(lowerunion)/len(dftemp.index)
            # print("new gamma===",gamatemp)
            if gamatemp > gama:
                gama = gamatemp
                T = tempT.copy()
            # print("new t=",T)
        return T

    def hReduced(self):
        red = self.reduct()
        red.append(self.y)
        dfred = self.dataframe[red]
        return dfred

    def vReduced(self):
        hred = self.hReduced()
        vred = hred.drop_duplicates()
        return vred


# Decision Rule
* A decision rule is a set of conditions that classify records.
* It is a **custom build classification model** build using concepts of Rough Set Theory.

* **class decisionRuleGen** has functions:
* * `rule_gen(train,y)` : generates a ruleset on a given training pandas dataframe and class column name "y", for each class a list of rules is produced, each rule consist of list of dicts.
* * `predict(testset)` : generates a 2D array of predicted class of classless test dataframe, by matching the the object with the rules using `matches()`, values in dicts of each rule are compared in OR condition and all dicts in a rule is compared in AND, if any of the rule match the corrosponding class is returned.  

In [ ]:
class decisionRuleGen:
    rules=dict()
    def finddiff(self,j,k):
        res=dict()
        for i in range(len(j)):
            if(j.iloc[i]!=k.iloc[i]):
                res[i]=j.iloc[i]

        return res


    def printmat(self,mat):
        for i in mat:
            print(i)
            print("\n")

    def rule_gen(self,train,y):
        global rules
        classlessdataframe=train.drop([y],axis=1)
        decisionmatrix=dict()
        for i in list(train[y].unique()):
            dmy=classlessdataframe[train[y]==i]
            dmx=classlessdataframe[train[y]!=i]
            unique=list()
            for index,rowsy in dmy.iterrows():
                unique1=list()
                seen=[]
                for index,rowsx in dmx.iterrows():
                    unique11=self.finddiff(rowsy,rowsx)
                    unique1.append(unique11.copy())
                    # print("1st : ",unique11)
                    # if [x for x in unique11 if x in seen]==[]:
                    #     unique1.append(unique11.copy())
                    #     for x in unique11:
                    #         seen.append(x)
                    #     print(seen)
                # print("2nd : ",unique1)
                unique.append(unique1.copy())
            # print("finally : ",unique)
            decisionmatrix[i]=unique.copy()
        # print(decisionmatrix)
        rules=decisionmatrix
        return decisionmatrix


    def matches(self,obj,ruleset):
            # print(obj)
            for i in ruleset:               #OR
                # print("i ",i)               #OR
                for j in i:                 #AND
                    # print("j ",j)           #AND
                    flag=0                  #AND
                    match=1                 #AND
                    for k in j:
                        # print(f"testing for {k} {j[k]},{obj[k]}")            #OR
                        if obj.iloc[k]==j[k]:    #OR
                            # print(f"---match for {k} ({i.index(j)}/{len(i)})")
                            flag=1          #OR
                            break           #OR
                    if flag==0:             #AND
                        # print(f"------didnt match for {j}")
                        match=0             #AND
                        break               #AND
            if match==0:                #OR
                return False
            # print(f"--------------finally {obj} match for {i}")
            return True

    def predict(self,testset):
            global rules
            res=list()
            # print(len(testset))
            for index,i in testset.iterrows():
                rest=list()
                for j in rules:
                    # print(rules[j])
                    if(self.matches(i,rules[j])):
                        # print(j)
                        rest.append(j)
                        # break
                res.append(rest)
                # print(index, " done ", res)
            return res




# Naive Bayes
* Describes the probability of an event, based on prior knowledge of conditions that might be related to the event.
* It is a **custom build classifier model** built using Bayes Theorum of probablity.

* **class NaieveBayes** has functions:
* * `train(d,y)` : sets data for running probablity of object based on.
* * `predict(y)` : predicts class of each object in the test dataframe and returnes a 2D list of predictions, predictions are made based on Bayes Theorum of finding probablity of and event occouring, provided another event has already occoured, for every object in classless test set, every attribute is considered and probablity of that attribute happening to be in each class is computed. all the attributs probablity is multiplied and finally divided by the probablity of a class happening. Finally whichever class has the highest probablity given the object's attributes is returned.

In [ ]:
class NaieveBayes:
    data=[]
    y=""
    def train(self,d,y):
        self.data=d
        self.y=y
    def prob(self,data,y):
        return len(data[data==y])/len(data)
    def probnb(self,x,y,classlen):
        if(x!=0):
            return(x/y)
        else:
            return(1/(y+classlen))
    def predict(self,y):
        pci=dict()
        count=dict()
        totcount=len(self.data)
        for yunique in self.data[self.y].unique():
            pci[yunique]=self.prob(self.data[self.y],yunique)
            count[yunique]=len(self.data[self.data[self.y]==yunique])
        # print(pci)
        # print(count)
        # print(totcount)
        # pxci=dict()
        cols=list(y.columns)
        res=list()
        for index,row in y.iterrows():
            pximul={x:1 for x in self.data[self.y].unique()}
            for j in cols:
                for yunique in count:
                    # print(yunique,row[j])
                    # self.probnb(j,(self.data[self.data[j]==row[j]),count[yunique])
                    dftemp=self.data[self.data[j]==row[j]]
                    # print(row[j],yunique)
                    temppxi=self.probnb(len(dftemp[dftemp[self.y]==yunique]),count[yunique],len(count))
                    # print(temppxi)
                    pximul[yunique]*=temppxi
                    # print(yunique,pximul[yunique])
            # print(pximul)
            resdic=dict()
            for i in pximul:
                resdic[i]=pximul[i]*pci[i]
            # print("Probablity for object no.:",index," belonging to:",resdic,end=" ")

            inverse = [(value, key) for key, value in resdic.items()]
            # print(max(inverse)[1])
            res.append([max(inverse)[1]])
        return res





# Decision Tree
* The decision tree classifier creates the classification model by building a decision tree. Each node in the tree specifies a test on an attribute, each branch descending from that node corresponds to one of the possible values for that attribute.

* It is **classification model** that uses Gain Ratio for Attribute Selection

### Imports
* * `from sklearn.tree import DecisionTreeClassifier` : DecisionTreeClassifier is a classification model that is used to build a decision tree, based on Information System which is then used to determine the class of unclassified data.
* * `from sklearn.preprocessing import OneHotEncoder` : One hot encoding is a technique that we use to represent categorical variables as numerical values, as the *DecisionTreeClassifier* model works efficiently on numerical values. This method is used to eleminate string categorical values from the Information System.

* **class DecisionTree** has functions:
* * `train_and_predict(train_data, class_column, test_data)` : takes a training pandas dataframe, class column name and unclassified test pandas dataframe. The Model is trained using the training data after encoding the dataframe using One Hot Encoding technique, the test dataframe is also encoded with the same categorical values and is used for prediction. Returns a list of predicted values of the unclassified testset.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder

class DecisionTree:
    def dfconcat(self,lis,axis):
        df1=lis[0]
        df2=lis[1]
        if not df1.empty and not df2.empty:
            result_df = pd.concat([df1, df2], axis=0)
        elif not df1.empty:
            result_df = df1.copy()
        elif not df2.empty:
            result_df = df2.copy()
        else:
            result_df = pd.DataFrame()
        return result_df

    def train_and_predict(self,train_data, class_column, test_data):
        # Extract features and target variable from train data
        X_train = train_data.drop(columns=[class_column])
        y_train = train_data[class_column]

        # One-hot encode categorical columns in train data
        categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
        encoder = OneHotEncoder(drop='first', sparse=False,handle_unknown='ignore')
        X_train.reset_index(drop=True,inplace=True)
        X_train_encoded = pd.DataFrame(encoder.fit_transform(X_train[categorical_cols]))

        X_train = self.dfconcat([X_train.drop(columns=categorical_cols), X_train_encoded], axis=1)

        # Initialize DecisionTreeClassifier
        clf = DecisionTreeClassifier()

        # Train the classifier
        clf.fit(X_train, y_train)
        # One-hot encode categorical columns in test data
        test_data.reset_index(drop=True,inplace=True)
        X_test_encoded = pd.DataFrame(encoder.transform(test_data[categorical_cols]))
        X_test = self.dfconcat([test_data.drop(columns=categorical_cols), X_test_encoded], axis=1)

        # Make predictions on test data
        predictions = clf.predict(X_test)

        return predictions

# Example usage:
# dataframe = pd.read_csv("computerData.csv")
# print(dataframe)
# for i in dataframe:
#     if dataframe.dtypes[i]=="int64":
#         dataframe[i]=dataframe[i].astype(str)
# train_df = dataframe.iloc[:10,:]
# test_df = dataframe.iloc[10:,:].drop(columns=["class"])
# dt=DecisionTree()
# print(len(test_df))
# predictions = dt.train_and_predict(train_df, 'class', test_df)
# print(predictions)


# Printing Performance
**`printingAPRF(results,actual)`** : funtion used to calculate Accuracy, Precision, Recall and F1Score of model from predictions and actual results. This funtion returnes a dictionary containing 4 keys: Accuracy, Precision, Recall and F1Score and there corrosponding values

In [ ]:
def printingAPRF(results,actual):
    truepos=0
    trueneg=0
    falsepos=0
    falseneg=0
    actual=actual.tolist()
    # print(len(actual),len(results),results)
    for i in range(len(actual)):
        # print(i,actual[i],results[i],actual[i] in results[i])
        if actual[i] in results[i]:
            if 'n' in actual[i] or 'N' in actual[i]:
                trueneg+=1
            else:
                truepos+=1
        else:
            if 'n' in actual[i] or 'N' in actual[i]:
                falseneg+=1
            else:
                falsepos+=1
    temp=(trueneg+truepos+falseneg+falsepos)
    accuracy=((truepos+trueneg)/temp) if temp!=0 else 1
    # print("Accuracy : ",accuracy)
    temp=(truepos+falsepos)
    precision=(truepos/temp) if temp!=0 else 1
    # print("Precision : ",precision)
    temp=(truepos+falseneg)
    recall=(truepos/temp) if temp!=0 else 1
    # print("Recall : ",recall)
    temp=(precision+recall)
    f1score=((2*precision*recall)/temp) if temp!=0 else 1
    # print("F1 Score : ",f1score)
    return {"Accuracy":accuracy,"Precision":precision,"Recall":recall,"F1 Score":f1score}


# Training and Testing Funtions
* These funtions take classifide train and test data, and returnes a dict containing keys actual test,actual train, test result, and train result and there corrosponding values,
* * "actual test" contains the actual classes of the Test data
* * "actual train" contains the actual classes of the Train data
* * "test result" contains the predicted classes of the Test data
* * "train result" contains the predicted classes of the Train data

* **`decisionModelTester(test,train)`** : generate decision rules based on training data using the Decision Rule model, and predicts the class of unclassified test data.
* **`naiveBayesModelTester(test,train)`** : predicts the class of unclassified test data using classified test data as experience for implimenting Bayes Probablity Theorum.
* **`decisionModelTester(test,train)`** : generate decision tree based on training data using the Decision Tree model, and predicts the class of unclassified test data.


In [ ]:
def decisionModelTester(test,train):
    global y
    dr=decisionRuleGen()
    rules=dr.rule_gen(train,y)
    classlesstrain=train.drop([y],axis=1)
    classlesstest=test.drop([y],axis=1)
    testresults=dr.predict(classlesstest)
    trainresults=dr.predict(classlesstrain)
    # print("this is ttest[y]",test[y])
    actualtest=test[y]
    actualtrain=train[y]
    return {"actual test":actualtest,"actual train":actualtrain,"test result":testresults,"train result":trainresults}

def naiveBayesModelTester(test,train):
    global y
    dr=NaieveBayes()
    dr.train(train,y)
    classlesstrain=train.drop([y],axis=1)
    classlesstest=test.drop([y],axis=1)
    testresults=dr.predict(classlesstest)
    trainresults=dr.predict(classlesstrain)
    actualtest=test[y]
    actualtrain=train[y]
    return {"actual test":actualtest,"actual train":actualtrain,"test result":testresults,"train result":trainresults}

def treeModelTester(test,train):
    global y
    dr=DecisionTree()
    classlesstrain=train.drop([y],axis=1)
    classlesstest=test.drop([y],axis=1)
    testresults=dr.train_and_predict(train, y, classlesstest)
    testresults=[[x] for x in testresults]
    trainresults=dr.train_and_predict(train, y, classlesstrain)
    trainresults=[[x] for x in trainresults]
    actualtest=test[y]
    actualtrain=train[y]
    return {"actual test":actualtest,"actual train":actualtrain,"test result":testresults,"train result":trainresults}



# Data Set
### We use *`diabetes_dataset.csv`* for testing and training our models and compairing there classification ability
* The index column is dropped and the final shape is printed

In [ ]:
originaldataframe=pd.read_csv("diabetes_dataset.csv")
originaldataframe.reset_index(drop=True,inplace=True)
print(originaldataframe.head(),originaldataframe.shape)

   Age Gender Polyuria Polydipsia sudden weight loss weakness Polyphagia  \
0   40   Male       No        Yes                 No      Yes         No   
1   60   Male       No         No                 No      Yes         No   
2   50   Male      Yes         No                 No      Yes        Yes   
3   50   Male       No         No                Yes      Yes        Yes   
4   60   Male      Yes        Yes                Yes      Yes        Yes   

  Genital thrush visual blurring Itching Irritability delayed healing  \
0             No              No     Yes           No             Yes   
1             No             Yes      No           No              No   
2             No              No     Yes           No             Yes   
3            Yes              No     Yes           No             Yes   
4             No             Yes     Yes          Yes             Yes   

  partial paresis muscle stiffness Alopecia Obesity     class  
0              No              Yes      

# Reduced Data Set
* Decision class name - "class" is stored in variable `y`,
* The dataframe is reduced using concepts of Rough Set Theory. `redandcode` is an object of **class reductandcore** which is fitted with original dataframe and class name, `.vReduced()` funtion returnes the reduced dataframe.
* Shape of reduced dataframe is printed

In [ ]:
y="class"
redandcore=reductandcore(dataframe=originaldataframe,y=y)
dataframe=redandcore.vReduced()
print(dataframe.head(),"\nshape of reduced data=",dataframe.shape)

  sudden weight loss weakness Gender Obesity Polydipsia  Age Polyuria  \
0                 No      Yes   Male     Yes        Yes   40       No   
1                 No      Yes   Male      No         No   60       No   
2                 No      Yes   Male      No         No   50      Yes   
3                Yes      Yes   Male      No         No   50       No   
4                Yes      Yes   Male     Yes        Yes   60      Yes   

  partial paresis Polyphagia visual blurring Alopecia Itching     class  
0              No         No              No      Yes     Yes  Positive  
1             Yes         No             Yes      Yes      No  Positive  
2              No        Yes              No      Yes     Yes  Positive  
3              No        Yes              No       No     Yes  Positive  
4             Yes        Yes             Yes      Yes     Yes  Positive   
shape of reduced data= (219, 13)


# K Fold cross validation

* Variable `k` is used to store the number of foldes used to cross verify the training and testing data
* Dataframe is split into corrosponing ratio (for k=5, test:train = 20:80)
* The test and training datas are passed on to training and testing funtions
* The predicted and actual class values returned from the testiing funtion is passed to the `printingAPRF()` funtion which returnes the performance scores of the models.
* The Accuracy, Precision, Recall and F1Score of the model are printed on both the test and training data.

In [ ]:
k=5
no_of_rows=len(dataframe)
splitsize=int(no_of_rows/k)

score={"test":{"decisionrule":dict(),"naievebayes":dict(),"decisiontree":dict()},"train":{"decisionrule":dict(),"naievebayes":dict(),"decisiontree":dict()}}
for i in dataframe:
    if dataframe.dtypes[i]=="int64":
        dataframe[i]=dataframe[i].astype(str)
for i in range(k):
    test=dataframe.iloc[i*splitsize:(i+1)*splitsize,:]
    train1=dataframe.iloc[0:i*splitsize,:]
    train2=dataframe.iloc[(i+1)*splitsize:,:]
    train=pd.concat([train1,train2])
    print(f"-----------------------------------------\nFold -- {i+1}")
    print(f"test shape {test.shape}, train shape {train.shape} \n")

    decisionrule=decisionModelTester(test,train)
    score["test"]["decisionrule"][i]=printingAPRF(decisionrule["test result"],decisionrule["actual test"])
    score["train"]["decisionrule"][i]=printingAPRF(decisionrule["train result"],decisionrule["actual train"])
    print("DECISION RULE")
    print("Test Score:")
    print(score["test"]["decisionrule"][i])
    print("-------------------------")
    print("Train Score:")
    print(score["train"]["decisionrule"][i])
    print("------------------------------------------------------------------\n")

    naievebayes=naiveBayesModelTester(test,train)
    score["test"]["naievebayes"][i]=printingAPRF(naievebayes["test result"],naievebayes["actual test"])
    score["train"]["naievebayes"][i]=printingAPRF(naievebayes["train result"],naievebayes["actual train"])
    print("NAIEVE BAYES")
    print("Test Score:")
    print(score["test"]["naievebayes"][i])
    print("-------------------------")
    print("Train Score:")
    print(score["train"]["naievebayes"][i])
    print("------------------------------------------------------------------\n")

    decisiontree=treeModelTester(test,train)
    score["test"]["decisiontree"][i]=printingAPRF(decisiontree["test result"],decisiontree["actual test"])
    score["train"]["decisiontree"][i]=printingAPRF(decisiontree["train result"],decisiontree["actual train"])
    print("DECISION TREE")
    print("Test Score:")
    print(score["test"]["decisiontree"][i])
    print("-------------------------")
    print("Train Score:")
    print(score["train"]["decisiontree"][i])
    print("------------------------------------------------------------------\n\n")


-----------------------------------------
Fold -- 1
test shape (43, 13), train shape (176, 13) 

DECISION RULE
Test Score:
{'Accuracy': 0.8837209302325582, 'Precision': 0.8837209302325582, 'Recall': 1.0, 'F1 Score': 0.9382716049382717}
-------------------------
Train Score:
{'Accuracy': 0.8522727272727273, 'Precision': 0.9495798319327731, 'Recall': 0.849624060150376, 'F1 Score': 0.8968253968253969}
------------------------------------------------------------------

NAIEVE BAYES
Test Score:
{'Accuracy': 0.813953488372093, 'Precision': 0.813953488372093, 'Recall': 1.0, 'F1 Score': 0.8974358974358974}
-------------------------
Train Score:
{'Accuracy': 0.8522727272727273, 'Precision': 0.8487394957983193, 'Recall': 0.926605504587156, 'F1 Score': 0.8859649122807017}
------------------------------------------------------------------

DECISION TREE
Test Score:
{'Accuracy': 0.8837209302325582, 'Precision': 0.8837209302325582, 'Recall': 1.0, 'F1 Score': 0.9382716049382717}
---------------------

# CONCLUSION
## The average performance of each of the model based on seen and unseen data is printed:

* It is seen that the Decision Rule model performs best on unseen data, Accuracy, Precision and F1Score wise.
* Naive Bayes classifier model perform next best on unseen data, but has the highest Recall.
* The Decision Tree classifier model performs the worst among the three on unseen data, Accuracy, Precision, Recall and F1score wise.

* On the other hand on seen data, the Decision Tree classifier model performs the best among the three, Accuracy, Precision, Recall and F1score wise.
* Naive Bayes classifier model perform next best on seen data, but has the lowest Precision.
* The Decision Rule model performs worst on seen data, Accuracy, Recall and F1score wise.


In [ ]:
def avgscore(dic):
    res={'Accuracy': 0, 'Precision': 0, 'Recall': 0, 'F1 Score': 0}
    for i in dic:
        for j in dic[i]:
            res[j]=res[j]+dic[i][j]
    for i in res:
        res[i]=res[i]/len(dic)
    return res

print("AVERGE PERFORMANCE\n---------------------------------------------")
print("ON UNSEEN DATA:")
print("     DECISION RULE")
print("         ",avgscore(score["test"]["decisionrule"]))
print("     NAIEVE BAYES")
print("         ",avgscore(score["test"]["naievebayes"]))
print("     DECISION TREE")
print("         ",avgscore(score["test"]["decisiontree"]))
print("\nON SEEN DATA:")
print("     DECISION RULE")
print("         ",avgscore(score["train"]["decisionrule"]))
print("     NAIEVE BAYES")
print("         ",avgscore(score["train"]["naievebayes"]))
print("     DECISION TREE")
print("         ",avgscore(score["train"]["decisiontree"]))



AVERGE PERFORMANCE
---------------------------------------------
ON UNSEEN DATA:
     DECISION RULE
          {'Accuracy': 0.8372093023255814, 'Precision': 0.9509966777408639, 'Recall': 0.7651282051282051, 'F1 Score': 0.786383364844003}
     NAIEVE BAYES
          {'Accuracy': 0.7627906976744185, 'Precision': 0.8669435215946845, 'Recall': 0.8018518518518519, 'F1 Score': 0.749886062081184}
     DECISION TREE
          {'Accuracy': 0.6976744186046513, 'Precision': 0.8498338870431894, 'Recall': 0.7606481481481481, 'F1 Score': 0.7182616548302823}

ON SEEN DATA:
     DECISION RULE
          {'Accuracy': 0.8579545454545455, 'Precision': 0.9414735670387557, 'Recall': 0.8715721239056003, 'F1 Score': 0.9048815142828165}
     NAIEVE BAYES
          {'Accuracy': 0.8727272727272727, 'Precision': 0.8795857895396966, 'Recall': 0.9384579229558467, 'F1 Score': 0.9079656604290474}
     DECISION TREE
          {'Accuracy': 0.996590909090909, 'Precision': 0.995146118148752, 'Recall': 1.0, 'F1 Score': 0.9